1. Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import numpy as np
from sklearn.metrics import f1_score
from PIL import Image
import matplotlib.pyplot as plt


2. Check for GPU

In [2]:
# Check for GPU (CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


3. Dataset and DataLoader

Path Setup

In [3]:
data_dir = r"C:/Users/GAMING/Documents/dataset"


Transformations

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


Load Dataset

In [5]:
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

Split into Train/Validation

In [6]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


4. Define CNN Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CNN().to(device)


5. Define Loss Function and Optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


6. Training and Validation

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        true_labels = []
        predicted_labels = []
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())
        
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        f1 = f1_score(true_labels, predicted_labels, average='binary')
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, F1 Score: {f1:.4f}")


7. Train the Model

In [10]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Epoch [1/10], Loss: 0.8604, Accuracy: 61.50%, Val Loss: 0.3176, Val Accuracy: 84.31%, F1 Score: 0.8000
Epoch [2/10], Loss: 0.2986, Accuracy: 84.50%, Val Loss: 0.1340, Val Accuracy: 94.12%, F1 Score: 0.9388
Epoch [3/10], Loss: 0.3202, Accuracy: 91.00%, Val Loss: 0.0999, Val Accuracy: 96.08%, F1 Score: 0.9565
Epoch [4/10], Loss: 0.1272, Accuracy: 94.00%, Val Loss: 0.0686, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [5/10], Loss: 0.1150, Accuracy: 95.00%, Val Loss: 0.0402, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [6/10], Loss: 0.0626, Accuracy: 97.00%, Val Loss: 0.0423, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [7/10], Loss: 0.0247, Accuracy: 100.00%, Val Loss: 0.0736, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [8/10], Loss: 0.0365, Accuracy: 98.00%, Val Loss: 0.0563, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [9/10], Loss: 0.0134, Accuracy: 99.50%, Val Loss: 0.0214, Val Accuracy: 98.04%, F1 Score: 0.9787
Epoch [10/10], Loss: 0.0229, Accuracy: 99.00%, Val Loss: 0.0389, Val Acc

8. Save the Model

In [11]:
torch.save(model.state_dict(), "covid_model.pth")
print("Model saved!")


Model saved!


9. Evaluation

Path Setup

In [12]:
eval_dir = r"C:/Users/GAMING/Documents/dataset/"
eval_dataset = datasets.ImageFolder(root=eval_dir, transform=transform)
eval_loader = DataLoader(eval_dataset, batch_size=1, shuffle=False)


Load Model for Evaluation

In [13]:
model.load_state_dict(torch.load("covid_model.pth"))
model.eval()


C:\Users\GAMING\AppData\Local\Temp\ipykernel_23832\3922468033.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("covid_model.pth"))


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

Inference and Save Results


In [14]:
results = []
true_labels = []
predicted_labels = []

with torch.no_grad():
    for i, (image, label) in enumerate(eval_loader):
        image = image.to(device)
        output = model(image)
        _, predicted = torch.max(output, 1)
        
        image_name = os.path.basename(eval_dataset.samples[i][0])
        results.append(f"{image_name} {predicted.item()}")
        
        true_labels.append(label.item())
        predicted_labels.append(predicted.item())

# Save results to result.txt
with open("result.txt", "w") as f:
    f.write("\n".join(results))

# Calculate F1 Score
f1 = f1_score(true_labels, predicted_labels, average='binary')
print(f"F1 Score on Evaluation Set: {f1:.4f}")


F1 Score on Evaluation Set: 0.9955
